# Data class example

In [48]:
import numpy as np
from skimage import io
import json
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models

from torchvision.datasets.utils import download_file_from_google_drive
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader,Dataset
from PIL import Image
import os
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
from os import listdir
from os.path import isfile, join
import pandas as pd

output_feat = 2048

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Utility functions

In [14]:
def get_filepaths(directory, format):
    """
    This function will generate the file names in a directory 
    tree by walking the tree either top-down or bottom-up. For each 
    directory in the tree rooted at directory top (including top itself), 
    it yields a 3-tuple (dirpath, dirnames, filenames).
    """
    file_paths = []  # List which will store all of the full filepaths.

    # Walk the tree.
    for root, directories, files in os.walk(directory):
        for filename in files:
            if format in filename: 
            # Join the two strings in order to form the full filepath.
                filepath = os.path.join(root, filename)
                file_paths.append(filepath)  # Add it to the list.
            else:
                pass
    return file_paths  # Self-explanatory.

### Dataloader demo

In [62]:
train_transform = transforms.Compose([
      transforms.ToTensor()               
])

valid_transform = transforms.Compose([
    transforms.ToTensor(),           
])

class DengueSet_loader(Dataset):
    def __init__(self, root, transform = None):
        self.root = root
        self.imgs = list(sorted(get_filepaths(root, ".tiff")))
        self.targets = list(sorted(get_filepaths(root, ".json")))
    def __len__(self):
        return len(self.imgs)
    def __getitem__(self, ind):
        image_path = self.imgs[ind]
        target_path = self.targets[ind]
        # Read data
        image = io.imread(image_path)
        metadata = json.load(open(target_path))
        
        
        #if self.transform:
        #  image = self.transform(image)
        
        return image, metadata# image_path,target_path
  
root = "./DATASET/"
train_set = DengueSet_loader(root, train_transform)
loader = DataLoader(train_set, batch_size=1)
print(len(loader))
cont = 0
df = []
for x, y in loader:
        print(x.shape,y)
        cont+=1
        if cont==5:
                break
        

5
torch.Size([1, 745, 748, 12]) {'image_path': ['DATASET/images/76001/image_2016-01-03.tiff'], 'municipality_code': tensor([76001]), 'date': tensor([201601]), 'labels': {'cases': tensor([408]), 'binary_classification': tensor([1]), 'incidence_ratio': tensor([0]), 'multilabel': tensor([0])}, 'metadata': {'environmental_data': {'temperature': [tensor([23.8408], dtype=torch.float64)], 'precipitation': [tensor([13.8939], dtype=torch.float64)], 'elevation': tensor([1075.], dtype=torch.float64)}, 'socioeconomic_data': {'Population': tensor([2208181]), 'Age0-4(%)': tensor([5.3700], dtype=torch.float64), 'Age5-14(%)': tensor([24.7300], dtype=torch.float64), 'Age>30(%)': tensor([57.5200], dtype=torch.float64), 'AfrocolombianPopulation(%)': tensor([14.4400], dtype=torch.float64), 'IndianPopulation(%)': tensor([0.5200], dtype=torch.float64), 'PeoplewithDisabilities(%)': tensor([9.6700], dtype=torch.float64), 'Peoplewhocannotreadorwrite(%)': tensor([3.0200], dtype=torch.float64), 'Secondary/Higher

In [67]:
df = []
df = pd.DataFrame(y["labels"])
df[["temperature", "precipitation", "elevation"]]=pd.DataFrame(y["metadata"]["environmental_data"])
df[['Population', 'Age0-4(%)', 'Age5-14(%)', 'Age>30(%)',
            'AfrocolombianPopulation(%)', 'IndianPopulation(%)',
            'PeoplewithDisabilities(%)', 'Peoplewhocannotreadorwrite(%)',
            'Secondary/HigherEducation(%)', 'Employedpopulation(%)',
            'Unemployedpopulation(%)', 'Peopledoinghousework(%)', 'Men(%)',
            'Women(%)', 'Householdswithoutwateraccess(%)',
            'Householdswithoutinternetaccess(%)', 'Buildingstratification1(%)',
            'Buildingstratification2(%)', 'Buildingstratification3(%)',
            'Buildingstratification4(%)', 'Buildingstratification5(%)',
            'Buildingstratification6(%)', 'NumberofhospitalsperKm2',
            'NumberofhousesperKm2']]= pd.DataFrame(y["metadata"]["socioeconomic_data"])

In [68]:
df

,cases,binary_classification,incidence_ratio,multilabel,temperature,precipitation,elevation,Population,Age0-4(%),Age5-14(%),...,Householdswithoutwateraccess(%),Householdswithoutinternetaccess(%),Buildingstratification1(%),Buildingstratification2(%),Buildingstratification3(%),Buildingstratification4(%),Buildingstratification5(%),Buildingstratification6(%),NumberofhospitalsperKm2,NumberofhousesperKm2
0,745,1,0,1,"[tensor(26.8909, dtype=torch.float64)]","[tensor(0.6262, dtype=torch.float64)]",1075.0,2208181,5.37,24.73,...,0.88,23.99,17.3906,23.1654,27.317,9.2994,6.5725,2.3033,1.053381,1064.918149
